In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

#Copiando banco de dados do meu Drive para o colab local
!cp '/content/drive/MyDrive/ALLM/STORE_db.DB' '/content'
!cp '/content/drive/MyDrive/ALLM/importDB.py' '/content'
!cp '/content/drive/MyDrive/ALLM/chromaDBconsultClass.py' '/content'
!cp '/content/drive/MyDrive/ALLM/Llama2FineTuned.py' '/content'
!cp '/content/drive/MyDrive/ALLM/ChatGPT.py' '/content'


In [4]:
!pip install langchain
!pip install sentence_transformers
!pip install chromadb
!pip install transformers
!pip install accelerate
!pip install einops
!pip install xformers
!pip install bitsandbytes
!pip install faiss-gpu


# !pip install pydantic-setting

!pip install pydantic

!pip install flask
!pip install flask-ngrok
!pip install pyngrok

!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=106e0069f505fdfa309fa77be7411cecd73d69147f2560ee2673ec49143ec14a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transfor

In [6]:
!ngrok authtoken '34523452345345'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [1]:
from importDB import CRUDSQLite
from chromaDBconsultClass import ProductsDB
import json
from Llama2FineTuned import FineTunedModel
from ChatGPT import ChamadaGPT

In [2]:
w = ProductsDB()
# llm = FineTunedModel()
llm = ChamadaGPT()

def getResult(userID,query):

  db = CRUDSQLite()


  # Consulta pedidos pelo ID do usuario
  resultConsultaDB = db.get_order_by_user_id_jsonFormat(userID)


  # if resultConsultaDB:
  #   print(resultConsultaDB)
  # else:
  #   print("Registro não encontrado.")

  db.close_connection()


  resultadoChromaDB = w.searchDoc(query)
  # print(resultadoChromaDB)
  k = 0
  for i in resultadoChromaDB:
    i =  json.loads(i)
    del i["id"]
    del i["category"]
    del i["Usd-price"]
    del i["sub_category"]
    del i["average_rating"]
    del i["brand"]
    # del i["product_details"]
    # del i["description"]
    i['link'] = i.pop('url')
    resultadoChromaDB[k] = i
    k = k+1

  # prompt = "context: client orders: " + json.dumps(resultConsultaDB) + "\n --- \n estes são os produtos que a loja que você trabalha possue. Indique para o usuário colocando o url: " + str(resultadoChromaDB) + "\n \n " #user question: " + query

  # print(prompt)

  #aqui deve ser feita a consulta para a LLM
  resultadoConsultaLLM = llm.getLLmResponse(json.dumps(resultConsultaDB),str(resultadoChromaDB), query)


  return resultadoConsultaLLM
  # return prompt


In [3]:
# getResult("ffffe8b65bbe3087b653a978c870db99","i want blue t-shirt")


{"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": "pretend to be an online store clerk. Only give products informations that are on store product list or information about customers orders or information about the store politics"}, {"role": "system", "content": "Store product list: [{'description': 'True Blue Men Blue Casual shirts', 'product_details': \"[{'Pack of': '1'}, {'Style Code': '2031733379'}, {'Fit': 'Regular'}, {'Fabric': 'Cotton Blend'}, {'Sleeve': 'Full Sleeve'}, {'Pattern': 'Printed'}, {'Reversible': 'No'}, {'Collar': 'Button Down'}, {'Color': 'Blue'}, {'Fabric Care': 'Gentle Machine Wash'}, {'Suitable For': 'Western Wear'}, {'Hem': 'Straight'}]\", 'title': 'Men Regular Fit Printed Button Down Collar Casual Shirt', 'link': 'https://www.flipkart.com/true-blue-men-printed-casual-shirt/p/itmf98h7smtb57yz?pid=SHTF98GZRZEXFJN3&lid=LSTSHTF98GZRZEXFJN3FAGX53&marketplace=FLIPKART&srno=b_5_175&otracker=browse&fm=organic&iid=a780aa60-fb38-4d5f-8a4a-c3fdd28b8792.

"I apologize, but I cannot see any blue t-shirts on our store product list. However, we do have some blue casual shirts available. Here are a few options:\n\n1. Men Regular Fit Printed Button Down Collar Casual Shirt - This shirt comes in a blue color and is made of a cotton blend fabric. It has a full sleeve and a printed pattern. You can find more details and purchase it [here](https://www.flipkart.com/true-blue-men-printed-casual-shirt/p/itmf98h7smtb57yz?pid=SHTF98GZRZEXFJN3&lid=LSTSHTF98GZRZEXFJN3FAGX53&marketplace=FLIPKART&srno=b_5_175&otracker=browse&fm=organic&iid=a780aa60-fb38-4d5f-8a4a-c3fdd28b8792.SHTF98GZRZEXFJN3.SEARCH&ssid=s84hk6dkkw0000001612413057370).\n\n2. Men Slim Fit Printed Cut Away Collar Casual Shirt - This blue shirt has a slim fit and is made of a cotton blend fabric. It features a 3/4th sleeve and a printed pattern. You can find more details and purchase it [here](https://www.flipkart.com/true-blue-men-printed-casual-shirt/p/itmaaaabc1d8c680?pid=SHTFM32PTZGZDZA

In [18]:
from flask import Flask
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/<query>")
def home(query):

    result = getResult("ffffe8b65bbe3087b653a978c870db99",query)

    return result

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://0b69-35-234-41-204.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/Sep/2023 21:51:39] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Sep/2023 21:51:57] "GET /queria%20uma%20camisa%20azul HTTP/1.1" 200 -


{
  "id": "chatcmpl-7vuj6GRkvnZQBmgiVtK0ekVSEUMoL",
  "object": "chat.completion",
  "created": 1694037108,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Claro! Aqui est\u00e3o algumas op\u00e7\u00f5es de camisas azuis dispon\u00edveis em nossa loja online:\n\n1. Camisa Casual Azul - [Link](https://www.examplestore.com/camisa-azul-casual)\n2. Camisa Social Azul - [Link](https://www.examplestore.com/camisa-azul-social)\n3. Camisa Polo Azul - [Link](https://www.examplestore.com/camisa-azul-polo)\n\nPor favor, clique nos links para ver mais detalhes sobre cada camisa e fazer sua compra. Se tiver alguma outra prefer\u00eancia ou d\u00favida, estou aqui para ajudar!"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 679,
    "completion_tokens": 140,
    "total_tokens": 819
  }
}
